#### Импорт необходимых библиотек

In [25]:
import json
import os
import jiwer as jiwer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from vosk import Model, KaldiRecognizer
import wave
import numpy as np
from pydub import AudioSegment
from sklearn.feature_extraction.text import TfidfVectorizer
from fuzzywuzzy import process
from transformers import BertTokenizer, BertModel
import optuna
from gensim.models import Word2Vec
import librosa
import soundfile as sf

In [2]:
# Константы
DATA_DIR = '../data/train/'
DATA_DIR_FILES = [
    'hr_bot_clear',
    'hr_bot_noise',
    'hr_bot_synt'
]
ANNOTATION_DIR = '../data/train/annotation/'
ANNOTATION_FILES = [
    'hr_bot_clear.json',
    'hr_bot_noise.json',
    'hr_bot_synt.json'
]

VAL_DIR = '../data/val/luga/'  # Путь к валидационным данным
ANNOTATION_VAL_FILE = os.path.join(VAL_DIR, 'luga.json')

# Настройка Vosk модели для распознавания речи
MODEL_PATH = "../model/vosk_model"  # Путь к скачанной модели Vosk

In [3]:
# Настройка Vosk модели для распознавания речи
MODEL_PATH = "../model/vosk_model"
model = Model(MODEL_PATH)
print("Модель Vosk загружена успешно.")

Модель Vosk загружена успешно.


In [4]:
# Метки команд
_label = {
    0: "отказ",
    1: "отмена",
    2: "подтверждение",
    3: "начать осаживание",
    4: "осадить на (количество) вагон",
    5: "продолжаем осаживание",
    6: "зарядка тормозной магистрали",
    7: "вышел из межвагонного пространства",
    8: "продолжаем роспуск",
    9: "растянуть автосцепки",
    10: "протянуть на (количество) вагон",
    11: "отцепка",
    12: "назад на башмак",
    13: "захожу в межвагонное пространство",
    14: "остановка",
    15: "вперед на башмак",
    16: "сжать автосцепки",
    17: "назад с башмака",
    18: "тише",
    19: "вперед с башмака",
    20: "прекратить зарядку тормозной магистрали",
    21: "тормозить",
    22: "отпустить",
}

#### Объявление классов

In [5]:
# Датасет для классификации текста
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

In [6]:
# Нейронная сеть для классификации текста
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

#### Объявление функций

In [7]:
def prepare_word2vec(texts):
    tokenized_texts = [text.split() for text in texts]
    model = Word2Vec(tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)
    return model

In [8]:
def encode_with_bert(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)  # Усреднение по всем токенам

In [26]:
# Аугментация аудио
def augment_audio(file_path):
    # Загрузка аудио файла
    audio, sr = librosa.load(file_path, sr=None)

    # Добавление белого шума
    noise = np.random.randn(len(audio))
    augmented_audio = audio + 0.005 * noise

    # Изменение скорости
    augmented_audio_speed = librosa.effects.time_stretch(audio, rate=1.2)

    # Сохранение аугментированного аудио с добавлением шума
    augmented_file_path = file_path.replace('.wav', '_augmented.wav')
    sf.write(augmented_file_path, augmented_audio, sr)

    # Сохранение аугментированного аудио с изменённой скоростью
    augmented_file_path_speed = file_path.replace('.wav', '_augmented_speed.wav')
    sf.write(augmented_file_path_speed, augmented_audio_speed, sr)

    return [augmented_file_path, augmented_file_path_speed]



In [19]:
def correct_transcription(transcribed_text):
    # Используем fuzzywuzzy для нахождения ближайшей команды
    corrected_text, score = process.extractOne(transcribed_text, label.values())
    if score >= 80:  # Порог для принятия исправления
        return corrected_text
    return transcribed_text  # Если нет похожей команды, возвращаем оригинал

In [11]:
# Преобразование MP3 в WAV
def convert_mp3_to_wav(mp3_filepath):
    try:
        # Убедимся, что pydub правильно обрабатывает MP3
        if not mp3_filepath.endswith('.mp3'):
            print(f"Файл {mp3_filepath} не является MP3.")
            return None

        wav_filepath = mp3_filepath.replace('.mp3', '.wav')
        audio = AudioSegment.from_file(mp3_filepath)  # Используем from_file для универсальности
        audio.export(wav_filepath, format='wav')
        print(f"Файл {mp3_filepath} успешно конвертирован в WAV: {wav_filepath}")
        return wav_filepath
    except Exception as e:
        print(f"Ошибка при конвертации {mp3_filepath} в WAV: {e}")
        return None

In [12]:
# Функция для обработки аудиофайла
def transcribe_audio(audio_file, dir):
    wf = wave.open(f"{dir}/{audio_file}", "rb")
    rec = KaldiRecognizer(model, wf.getframerate())

    result_text = ""
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            result = json.loads(rec.Result())
            result_text += result.get("text", "") + " "

    final_result = json.loads(rec.FinalResult())
    result_text += final_result.get("text", "")
    print(f"Распознанный текст для {audio_file}: {result_text}")
    return result_text.strip()

In [13]:
# Функция классификации текста
def classify_text(text, classifier, tokenizer):
    # Преобразование текста в вектор
    text_vector = tokenizer.transform([text]).toarray()
    text_tensor = torch.tensor(text_vector, dtype=torch.float32)

    # Классификация текста
    with torch.no_grad():
        outputs = classifier(text_tensor)

    _, predicted_class = torch.max(outputs, 1)

    return predicted_class.item()

In [68]:
def process_audio_files(file_list, classifier, tokenizer, dir):
    results = []
    for file_info in file_list:
        # [{'file': file_name, 'id': file_id}, ...]
        print(f"pocessing {file_info['file']}")
        audio_file = file_info['file']
        audio_id = file_info['id']

        # Распознавание текста из аудио
        transcribed_text = transcribe_audio(audio_file, dir)

        # Классификация текста
        predicted_class = classify_text(transcribed_text, classifier, tokenizer)

        # Извлечение атрибута
        attribute = any(char.isdigit() for char in transcribed_text)

        result = {
            "audio_filepath": os.path.basename(audio_file),
            "id": audio_id,
            "text": transcribed_text,
            "category": predicted_class,
            "attribute": attribute
        }

        results.append(result)
        print(f"done {file_info['file']}")

    return results

#### Подготовка настроек модели, создание модели

In [15]:
# Пример классификатора и токенизатора
input_dim = 2  # Зависит от метода векторизации текста
hidden_dim = 55
output_dim = len(_label)  # Количество классов
classifier = TextClassifier(input_dim, hidden_dim, output_dim)

#### Подготовка данных для обучения

In [16]:
# Функция загрузки аннотаций
def load_annotations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [27]:
def load_dataset():
    audio_files = []
    texts = []
    labels = []

    for annotation_file, data_dir in zip(ANNOTATION_FILES, DATA_DIR_FILES):
        annotation_path = os.path.join(ANNOTATION_DIR, annotation_file)
        print(f"Загрузка аннотаций из: {annotation_path}")
        training_annotations = load_annotations(annotation_path)

        for annotation in training_annotations:
            audio_filepath = os.path.join(DATA_DIR, data_dir, annotation['audio_filepath'])
            # Проверка существования файла перед конвертацией
            if os.path.exists(audio_filepath):
                if audio_filepath.endswith('.mp3'):
                    audio_filepath = convert_mp3_to_wav(audio_filepath)

                augmented_files = augment_audio(audio_filepath)  # Аугментация аудио
                audio_files.extend(augmented_files)

            else:
                print(f"Файл {audio_filepath} не найден.")
                continue  # Пропустить, если файл не найден


            text = annotation['text']
            label = annotation['label']
            texts.extend([text] * len(augmented_files))  # Повторяем текст для каждого аугментированного файла
            labels.extend([label] * len(augmented_files))

    print("Загрузка датасета завершена.")
    return audio_files, texts, labels

    print("Загрузка датасета завершена.")
    return audio_files, texts, labels

print("Начало загрузки данных...")
audio_files, texts, labels = load_dataset()

Начало загрузки данных...
Загрузка аннотаций из: ../data/train/annotation/hr_bot_clear.json
Файл ../data/train/hr_bot_clear\6ca54494-76ff-11ee-8f2f-c09bf4619c03.mp3 успешно конвертирован в WAV: ../data/train/hr_bot_clear\6ca54494-76ff-11ee-8f2f-c09bf4619c03.wav
Файл ../data/train/hr_bot_clear\6f15d2de-76ff-11ee-94ec-c09bf4619c03.mp3 успешно конвертирован в WAV: ../data/train/hr_bot_clear\6f15d2de-76ff-11ee-94ec-c09bf4619c03.wav
Файл ../data/train/hr_bot_clear\4ae49614-76ff-11ee-b181-c09bf4619c03.mp3 успешно конвертирован в WAV: ../data/train/hr_bot_clear\4ae49614-76ff-11ee-b181-c09bf4619c03.wav
Файл ../data/train/hr_bot_clear\53e7f721-76ff-11ee-b51e-c09bf4619c03.mp3 успешно конвертирован в WAV: ../data/train/hr_bot_clear\53e7f721-76ff-11ee-b51e-c09bf4619c03.wav
Файл ../data/train/hr_bot_clear\72623174-76ff-11ee-9e59-c09bf4619c03.mp3 успешно конвертирован в WAV: ../data/train/hr_bot_clear\72623174-76ff-11ee-9e59-c09bf4619c03.wav
Файл ../data/train/hr_bot_clear\763bd8fe-76ff-11ee-a307-c0

In [64]:
# Разделение данных на тренировочные и тестовые
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

tokenizer = TfidfVectorizer(max_features=input_dim)

# Подготовка текстовых данных для классификации
train_vectors = tokenizer.fit_transform(train_texts).toarray()
test_vectors = tokenizer.transform(test_texts).toarray()

train_dataset = TextDataset(train_vectors, train_labels)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

In [65]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001, weight_decay=0.01)

val_reference_texts = []
val_audio_files = []

def train():
    for epoch in range(30):
        for texts_batch, labels_batch in train_loader:
            texts_batch = texts_batch.float()
            optimizer.zero_grad()
            outputs = classifier(texts_batch)
            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()

        # Оценка на валидационном наборе после каждой эпохи
        if (epoch + 1) % 5 == 0:  # Оценка каждые 5 эпох
            # val_hypothesis_texts = process_audio_files(val_audio_files, classifier, tokenizer, dir)

            # # Рассчитываем WER для валидационного набора
            # wer = jiwer.wer(val_reference_texts, val_hypothesis_texts)
            print(f"Эпоха {epoch + 1}")

In [66]:
train()

Эпоха 5
Эпоха 10
Эпоха 15
Эпоха 20
Эпоха 25
Эпоха 30


In [69]:
# директория, где лежат файлы, которые мы хотим обработать
result_dir = '../data/train/hr_bot_synt'

result_files = os.listdir(result_dir)

for i in range(len(result_files)):
    result_files[i] = {"file": result_files[i], "id": result_files[i][:-4]}

# Обработка файлов и получение результатов
transcription_results = process_audio_files(result_files, classifier, tokenizer, result_dir)

# Сохранение результатов в JSON
with open("transcriptions.json", "w", encoding="utf-8") as f:
    json.dump(transcription_results, f, ensure_ascii=False, indent=4)

pocessing 330e9fce-76ff-11ee-8ffb-c09bf4619c03_1.wav
Распознанный текст для 330e9fce-76ff-11ee-8ffb-c09bf4619c03_1.wav: 
done 330e9fce-76ff-11ee-8ffb-c09bf4619c03_1.wav
pocessing 330e9fce-76ff-11ee-8ffb-c09bf4619c03_1_augmented.wav
Распознанный текст для 330e9fce-76ff-11ee-8ffb-c09bf4619c03_1_augmented.wav: 
done 330e9fce-76ff-11ee-8ffb-c09bf4619c03_1_augmented.wav
pocessing 330e9fce-76ff-11ee-8ffb-c09bf4619c03_1_augmented_speed.wav
Распознанный текст для 330e9fce-76ff-11ee-8ffb-c09bf4619c03_1_augmented_speed.wav: 
done 330e9fce-76ff-11ee-8ffb-c09bf4619c03_1_augmented_speed.wav
pocessing 330e9fce-76ff-11ee-8ffb-c09bf4619c03_2.wav
Распознанный текст для 330e9fce-76ff-11ee-8ffb-c09bf4619c03_2.wav: 
done 330e9fce-76ff-11ee-8ffb-c09bf4619c03_2.wav
pocessing 330e9fce-76ff-11ee-8ffb-c09bf4619c03_2_augmented.wav
Распознанный текст для 330e9fce-76ff-11ee-8ffb-c09bf4619c03_2_augmented.wav: 
done 330e9fce-76ff-11ee-8ffb-c09bf4619c03_2_augmented.wav
pocessing 330e9fce-76ff-11ee-8ffb-c09bf4619c03_

KeyboardInterrupt: 

In [75]:
def extract_features(audio_filepath):
    try:
        # Загрузка аудиофайла
        signal, sr = librosa.load(audio_filepath, sr=None)
        # Извлечение MFCC
        mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
        return np.mean(mfccs.T, axis=0)
    except Exception as e:
        print(f"Ошибка при обработке {audio_filepath}: {e}")
        return None

In [73]:
# Функция для предсказания
def predict_command(audio_filepath):
    features = extract_features(audio_filepath)
    if features is not None:
        features = np.reshape(features, (1, -1))  # Изменяем размерность
        prediction = model.predict(features)
        predicted_label = np.argmax(prediction)
        return predicted_label
    return None

In [70]:
# Функция загрузки аннотаций
def load_annotations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [77]:
VAL_DIR = '../data/val/luga/'  # Путь к валидационным данным
ANNOTATION_VAL_FILE = os.path.join(VAL_DIR, 'luga.json')
from sklearn.model_selection import train_test_split
def evaluate_commands():
    annotations = load_annotations(ANNOTATION_VAL_FILE)
    correct_predictions = 0
    total_predictions = len(annotations)

    # Словарь для хранения статистики
    statistics = {
        'correct': {},
        'total': {},
        'accuracy': {}
    }

    for item in annotations:
        audio_filepath = item['audio_filepath']
        label = item['label']

        audio_path = os.path.join(VAL_DIR, audio_filepath)
        if os.path.exists(audio_path):
            prediction = predict_command(audio_path)
            print(f'Predicted: {prediction}, Actual: {label} for {audio_path}')

            if prediction == label:
                correct_predictions += 1

    accuracy = correct_predictions / total_predictions

    # Сохраняем точность в статистике
    statistics['accuracy'] = accuracy

    # Выводим статистику
    print(f'\nTotal Predictions: {total_predictions}')
    print(f'Correct Predictions: {correct_predictions}')
    print(f'Accuracy: {accuracy:.2f}')
    print('Detailed statistics per label:')
    for label in statistics['total']:
        total = statistics['total'][label]
        correct = statistics['correct'].get(label, 0)
        print(f'Label: {label}, Total: {total}, Correct: {correct}, Accuracy: {correct / total:.2f}' if total > 0 else f'Label: {label}, Total: {total}, Correct: {correct}')


evaluate_commands()

AttributeError: 'Model' object has no attribute 'predict'

In [28]:
# Использование BERT для контекстуальной векторизации
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [29]:
# Подготовка данных для Word2Vec
word2vec_model = prepare_word2vec(texts)
word2vec_vectors = np.array([word2vec_model.wv[text.split()].mean(axis=0) for text in texts])

# Подготовка TF-IDF векторов
tfidf_vectorizer = TfidfVectorizer(max_features=100)  # Максимальное количество признаков
tfidf_vectors = tfidf_vectorizer.fit_transform(texts).toarray()

# Пример использования BERT
bert_vectors = np.array([encode_with_bert(text).detach().numpy() for text in texts])

In [30]:
print("Разделение данных на тренировочные и тестовые...")
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

Разделение данных на тренировочные и тестовые...


#### Функция обучения модели (+ расчет метрик каждые 5 эпох)

In [44]:
# Обучение модели
def train_classifier(classifier, train_vectors, train_labels, epochs=30, batch_size=4):
    train_dataset = TextDataset(train_vectors, train_labels)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)

    for epoch in range(epochs):
        for texts_batch, labels_batch in train_loader:
            texts_batch = texts_batch.float()
            optimizer.zero_grad()
            outputs = classifier(texts_batch)
            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()

        if (epoch + 1) % 5 == 0:
            print(f"Эпоха {epoch + 1}: Потеря = {loss.item():.4f}")


In [45]:
# Обучение с использованием TF-IDF
print("Обучение модели с использованием TF-IDF...")
classifier_tfidf = TextClassifier(input_dim=tfidf_vectors.shape[1], hidden_dim=100, output_dim=len(set(labels)))
tfidf_vectors = tfidf_vectors[:len(train_labels)]
train_classifier(classifier_tfidf, tfidf_vectors, train_labels)


Обучение модели с использованием TF-IDF...
Эпоха 5: Потеря = 2.3473
Эпоха 10: Потеря = 1.7028
Эпоха 15: Потеря = 1.8457
Эпоха 20: Потеря = 1.8712
Эпоха 25: Потеря = 1.6510
Эпоха 30: Потеря = 2.5411


In [47]:
print(len(word2vec_vectors))
print(len(train_labels))

11630
9304


NameError: name 'train_loader' is not defined

In [48]:
# Обучение с использованием Word2Vec
print("Обучение модели с использованием Word2Vec...")
classifier_word2vec = TextClassifier(input_dim=100, hidden_dim=100, output_dim=len(set(labels)))  # 100 - размерность Word2Vec
word2vec_vectors = word2vec_vectors[:len(train_labels)]
train_classifier(classifier_word2vec, word2vec_vectors, train_labels)



Обучение модели с использованием Word2Vec...
Эпоха 5: Потеря = 1.5483
Эпоха 10: Потеря = 3.4141
Эпоха 15: Потеря = 1.7524
Эпоха 20: Потеря = 2.3348
Эпоха 25: Потеря = 1.5767
Эпоха 30: Потеря = 2.3949


In [50]:
print(len(bert_vectors))
print(len(train_labels))

11630
9304


In [52]:
# Обучение с использованием BERT
print("Обучение модели с использованием BERT...")
classifier_bert = TextClassifier(input_dim=768, hidden_dim=100, output_dim=len(set(labels)))  # 768 - размерность BERT
bert_vectors = bert_vectors[:len(train_labels)]
train_classifier(classifier_bert, bert_vectors, train_labels)


Обучение модели с использованием BERT...


RuntimeError: Expected target size [4, 21], got [4]

In [54]:
print("Обучение завершено.")

Обучение завершено.


#### Метрики производительности

In [58]:
# Validate audio files
audio_files_to_validate = [audio_file for audio_file in audio_files if audio_file.endswith('.wav')]
print(f"Starting audio validation for {len(audio_files_to_validate)} files...")

validation_results = process_audio_files(audio_files_to_validate, classifier, tokenizer)

Starting audio validation for 11630 files...


TypeError: string indices must be integers

In [ ]:
# Save results to JSON
results_file_path = '../data/results/validation_results.json'
with open(results_file_path, 'w', encoding='utf-8') as results_file:
    json.dump(validation_results, results_file, ensure_ascii=False, indent=4)

print(f"Validation results saved to {results_file_path}.")

In [59]:
# Вычисление WER
def calculate_wer(reference, hypothesis):
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    S = sum(1 for r, h in zip(reference_words, hypothesis_words) if r != h)
    D = len(reference_words) - len(hypothesis_words) if len(reference_words) > len(hypothesis_words) else 0
    I = len(hypothesis_words) - len(reference_words) if len(hypothesis_words) > len(reference_words) else 0
    N = len(reference_words)

    return (S + D + I) / N if N > 0 else 0

# Вычисление Mq
def calculate_mq(wer, f1_weighted):
    WERnorm = wer
    return 0.25 * (1 - WERnorm) + 0.75 * f1_weighted

#### Проверка модели на тестовых данных

In [61]:
def process_validation_files(val_dir, classifier, tokenizer):
    """Обрабатывает валидационные файлы и сохраняет результаты."""
    annotations = load_annotations(ANNOTATION_VAL_FILE)
    print(f"Загружено аннотаций: {len(annotations)}")

    result_files = []
    for annotation in annotations:
        audio_filepath = os.path.join(val_dir, annotation['audio_filepath'])
        if os.path.exists(audio_filepath):
            result_files.append({"file": audio_filepath, "id": annotation['id']})
        else:
            print(f"Файл {audio_filepath} не найден.")

    if result_files:
        # Обрабатываем аудиофайлы
        transcription_results = process_audio_files(result_files, classifier, tokenizer)

        # Сохранение результатов в JSON
        output_file = "transcriptions_validation.json"
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(transcription_results, f, ensure_ascii=False, indent=4)

        print(f"Результаты транскрипции сохранены в {output_file}")
    else:
        print("Нет доступных аудиофайлов для обработки.")

# Вызов функции для обработки валидационных файлов
print("Обработка валидационных файлов...")
process_validation_files(VAL_DIR, classifier, tokenizer)

Обработка валидационных файлов...
Загружено аннотаций: 610
Обработка файла: ../data/val/luga/21_11_2023/2023_11_21__09_54_58.wav


TypeError: transcribe_audio() missing 1 required positional argument: 'dir'